<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/achatbot_freeze_omni_voice_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install

In [ ]:
!cd /content && rm -rf achatbot && git clone --recursive https://github.com/ai-bot-pro/achatbot.git

In [3]:
%cd /content/achatbot


/content/achatbot


In [4]:
!git pull origin feat/voice-freeze-omni

From https://github.com/ai-bot-pro/achatbot
 * branch            feat/voice-freeze-omni -> FETCH_HEAD
Already up to date.


In [ ]:
!bash scripts/pypi_achatbot.sh dev

In [ ]:
!pip install -q "dist/achatbot-0.0.8.3-py3-none-any.whl[fastapi_bot_server,daily_room_audio_stream,livekit_room_audio_stream,agora_channel_audio_stream,silero_vad_analyzer,freeze_omni_voice_processor,queue]"


# download model

In [7]:
from google.colab import userdata
#NGROK_TOKEN=userdata.get('NGROK_TOKEN')
HF_TOKEN=userdata.get('HF_TOKEN')


In [ ]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential

In [ ]:
# FreezeOmni ckpt
# - audiollm: encoder and adapter
# - decoder: NAR decoder, AR decoder and codec decoder
!huggingface-cli download VITA-MLLM/Freeze-Omni --local-dir /content/ckpts/VITA-MLLM/Freeze-Omni --local-dir-use-symlinks False

In [ ]:
# textual llm Qwen2-7B-Instruct ckpt
!huggingface-cli download Qwen/Qwen2-7B-Instruct --local-dir /content/ckpts/Qwen/Qwen2-7B-Instruct --local-dir-use-symlinks False

# run webrtc bot

In [11]:
from google.colab import userdata
DAILY_API_KEY=userdata.get('DAILY_API_KEY')

LIVEKIT_URL=userdata.get('LIVEKIT_URL')
LIVEKIT_API_KEY=userdata.get('LIVEKIT_API_KEY')
LIVEKIT_API_SECRET=userdata.get('LIVEKIT_API_SECRET')


In [12]:
%cd /content/achatbot

/content/achatbot


In [13]:
!cat /content/daily_freeze_omni_voice_bot.json

cat: /content/daily_freeze_omni_voice_bot.json: No such file or directory


In [17]:
!DAILY_API_KEY=$DAILY_API_KEY \
  python -m src.cmd.bots.main -f /content/daily_freeze_omni_voice_bot.json

{
    "chat_bot_name": "DailyFreezeOmniVoiceBot",
    "config": {
        "vad": {
            "args": {
                "stop_secs": 0.7
            },
            "tag": "silero_vad_analyzer"
        },
        "voice_llm": {
            "args": {
                "args": {
                    "llm_path": "/content/ckpts/Qwen/Qwen2-7B-Instruct",
                    "model_path": "/content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints"
                }
            },
            "tag": "glm_voice_processor"
        }
    },
    "config_list": [],
    "room_manager": {
        "args": {
            "privacy": "public"
        },
        "tag": "daily_room"
    },
    "room_name": "chat-room",
    "room_url": "",
    "services": {
        "pipeline": "achatbot",
        "vad": "silero",
        "voice_llm": "freeze_omni"
    },
    "token": ""
}
2024-12-19 08:11:12,506 - chat-bot - INFO - /content/achatbot/src/cmd/bots/main.py:63 - <module> - bot_config:{'chat_bot_name': 'DailyFreezeOmniVoice

In [ ]:
from IPython.display import Audio
Audio(filename="/content/achatbot/records/bot_speak_2024-12-19_04-51-40.318.wav")


# inference

In [ ]:
!cd /content/achatbot/deps/FreezeOmni/ \
  && PYTHONPATH=./:$PYTHONPATH CUDA_VISIBLE_DEVICES=0 python bin/inference.py \
    --model_path /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints \
    --input_wav /content/hi.wav \
    --output_wav /content/answer_hi.wav \
    --llm_path /content/ckpts/Qwen/Qwen2-7B-Instruct \
    --top_p 0.8 \
    --top_k 20 \
    --temperature 0.8

[2024-12-19 04:06:06.420] Namespace(model_path='/content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints', llm_path='/content/ckpts/Qwen/Qwen2-7B-Instruct', top_k=20, top_p=0.8, temperature=0.8, input_wav='/content/hi.wav', output_wav='/content/answer_hi.wav')
[2024-12-19 04:06:09.289] the number of speech encoder params: 341.3681640625M
Loading checkpoint shards: 100% 4/4 [00:00<00:00, 14.88it/s]
[2024-12-19 04:06:10.170] the number of audio llm params: 7621.362796783447M
[2024-12-19 04:06:10.197] Checkpoint: loading from checkpoint /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints/audiollm/final.pt for GPU
[2024-12-19 04:06:15.638] reading a config file from /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints/decoder/model.json
[2024-12-19 04:06:17.692] the number of LLM2TTSCodecAR(NAR decoder and AR decoder(llama transformer blocks)) params: 157.08826065063477M
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favo

# server

In [ ]:
from google.colab import userdata
NGROK_TOKEN=userdata.get('NGROK_TOKEN')
#HF_TOKEN=userdata.get('HF_TOKEN')


In [ ]:
!pip install -q flask_socketio pyngrok

In [ ]:
!ngrok config add-authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!cd /content/achatbot/deps/FreezeOmni/ \
  && PYTHONPATH=./:$PYTHONPATH PYTHONDONTWRITEBYTECODE=1 CUDA_VISIBLE_DEVICES=0 python bin/server.py \
    --ip 127.0.0.1 \
    --port 8000 \
    --max_users 3 \
    --llm_exec_nums 1 \
    --timeout 180 \
    --model_path /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints \
    --llm_path /content/ckpts/Qwen/Qwen2-7B-Instruct \
    --top_p 0.8 \
    --top_k 20 \
    --temperature 0.8 \
    --ngrok

Namespace(model_path='/content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints', llm_path='/content/ckpts/Qwen/Qwen2-7B-Instruct', top_k=20, top_p=0.8, temperature=0.8, ip='127.0.0.1', port='8000', max_users=3, llm_exec_nums=1, timeout=180, ngrok=True, ssl=False)
[2024-12-19 03:06:10.872] the number of speech encoder params: 341.3681640625M
Loading checkpoint shards: 100% 4/4 [00:00<00:00, 11.21it/s]
[2024-12-19 03:06:11.954] the number of audio llm params: 7621.362796783447M
[2024-12-19 03:06:11.981] Checkpoint: loading from checkpoint /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints/audiollm/final.pt for GPU
[2024-12-19 03:06:17.588] reading a config file from /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints/decoder/model.json
[2024-12-19 03:06:17.589] reading a config file from /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints/decoder/model.json
[2024-12-19 03:06:17.590] reading a config file from /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints/decoder/model.json
[2024-12-19 03:06:23.026] t